References:

https://www.kaggle.com/code/abhijatt/association-rule-learning-apriori-market-baske

https://www.kaggle.com/code/emiliagalach/groceries-dataset-market-basket-analysis


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from apyori import apriori

In [4]:
ds = pd.read_csv('data/groceries_dataset.csv')
ds.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [5]:
ds = ds.drop(columns='Member_number')

In [9]:
#   The item 'bag' usually is selected alone, so we will remove it

ds = ds[ds['itemDescription'] != 'bags']

#   Now we will format the date

ds['Date'] = pd.to_datetime(ds['Date'], format='%d-%m-%Y')
ds

# Aggregate all the items sold on the same date into a single column.
ds = ds.groupby('Date')['itemDescription'].apply(list).reset_index()
ds.head()

,Date,itemDescription
0,2014-01-01,"[cleaner, sausage, tropical fruit, whole milk,..."
1,2014-01-02,"[beef, frankfurter, hamburger meat, soda, UHT-..."
2,2014-01-03,"[frankfurter, oil, beef, long life bakery prod..."
3,2014-01-04,"[ham, chocolate, instant coffee, specialty cho..."
4,2014-01-05,"[meat, hamburger meat, sausage, liver loaf, tr..."


In [11]:
transactions = []
for indexer in range(len(ds)):
    transactions.append(ds['itemDescription'].iloc[indexer])

In [ ]:
transactions

In [13]:
rules = apriori(transactions = transactions, min_support=0.00412087912, min_confidence = 0.6, min_lift = 1.9 , min_length = 2, max_length = 2)

In [14]:
results = list(rules)

# Putting the results well organised into a Pandas DataFrame
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

# Displaying the results non sorted
resultsinDataFrame.sort_values('Confidence', ascending=False)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,baby cosmetics,beef,0.004121,1.000000,2.128655
14,salad dressing,curd,0.008242,1.000000,1.978261
26,make up remover,margarine,0.006868,1.000000,2.116279
33,salad dressing,oil,0.006868,0.833333,3.111111
32,toilet cleaner,napkins,0.005495,0.800000,2.222901
13,rubbing alcohol,cream cheese,0.005495,0.800000,2.080000
9,make up remover,chocolate,0.005495,0.800000,1.987713
12,frozen chicken,cream cheese,0.005495,0.800000,2.080000
7,toilet cleaner,butter milk,0.005495,0.800000,2.554386
11,cookware,cream cheese,0.017857,0.764706,1.988235


In [15]:
resultFrame = resultsinDataFrame.iloc[:,0:-3]
resultFrame

# Aggregate all the items sold on the same date into a single column.
resultFrame = resultFrame.groupby('Left Hand Side')['Right Hand Side'].apply(list).reset_index()
resultFrame = resultFrame.rename(columns={'Left Hand Side': 'Product Purchased', 'Right Hand Side':'Also Purchased Along'})
resultFrame

,Product Purchased,Also Purchased Along
0,baby cosmetics,[beef]
1,cooking chocolate,[cream cheese ]
2,cookware,[cream cheese ]
3,decalcifier,[baking powder]
4,frozen chicken,"[butter milk, cream cheese , ham, misc. bevera..."
5,frozen fruits,"[napkins, sugar]"
6,make up remover,"[chocolate, detergent, margarine, seasonal pro..."
7,organic products,"[berries, butter milk]"
8,prosecco,[hygiene articles]
9,rubbing alcohol,"[cream cheese , frozen fish, frozen meals, mea..."
